In [ ]:
from numpy import *
from numpy.linalg import * # utile pour l'inversion de matrice et la diagonalisation
from matplotlib.pylab import *
from pandas import *

# Exercice 1 : diagonalisation

On considère la matrice
\begin{align*}
A=\begin{pmatrix}
1 & -7 & -2\\
-7 & 1 & 2\\
-2 & 2 & 10
\end{pmatrix}.
\end{align*}

**Question 1.** Diagonaliser la matrice avec la commande `eigh` (eigenvalue signifie valeur propre en anglais) qui renvoie les valeurs propres (à stocker dans un vecteur `valtemp`) et les vecteurs propres (à stocker dans une matrice `Ptemp`).



In [ ]:
# correction
A=array([[1,-7,-2],[-7,1,2],[-2,2,10]])
valtemp, Ptemp = eigh(A)
print(valtemp)
print(Ptemp)

**Question 2.** Vérifier que les matrices obtenues constituent bien une diagonalisation de A et que la base des vecteurs propres est bien orthonormée.

In [ ]:
# correction
# on écrit la matrice diagonale D  contenant les valeurs propres sur la diagonale
# et on calcule le produit Ptemp x D x inv(Ptemp)
D=diag(valtemp)
B=dot(dot(Ptemp,D),inv(Ptemp))
print(B)
# on calcule le produit de P.T et P
print(dot(Ptemp.T,Ptemp))

**Question 3.** Ranger par ordre décroissant les valeurs propres $\lambda_1\geq \lambda_2\geq \lambda_3$ dans un nouveau vecteur `val` et adapter la matrice de changement de base en une matrice `P` de sorte que 
$$A=P\begin{pmatrix}\lambda_1 & 0 & 0\\
0 & \lambda_2 & 0\\
0 & 0 & \lambda_3
\end{pmatrix}P^{-1}.$$

*Pour ordonner un vecteur `V` dans l'ordre croissant, il faut utiliser la commande `sort(V)`. Pour l'ordonner dans l'ordre décroissant, il faudra inverser les éléments de `sort(V)` de la manière suivante : `sort(V)[::-1]`.
On obtiendra les indices correspondant à l'ordonnancement croissant de 
`V` avec `argsort(V)`, et donc ceux correspondant à l'ordonnancement décroissant avec `argsort(V)[::-1]`.*

Vérifier que $A=P D P^{-1}$.

In [ ]:
# correction
ind = argsort(valtemp)[::-1] # indices de val ordonnées de manière décroissante
val = sort(valtemp)[::-1] # valeurs propres ordonnées
P = zeros((3,3))
for i in range(3):
    P[:,i]=Ptemp[:,ind[i]]
print(P)
print(dot(inv(P),dot(A,P))) # vérif que P^-1 A Q = D

**Question 4.** En utilisant la commande `trace`, calculer la trace de $A$ et vérifier que $\mathrm{tr}(A) = \lambda_1 +\lambda_2+\lambda_3$.

In [ ]:
# correction
trace(A)-sum(val)

# Exercice 2 : ACP sur les 5 étudiants

On reprend les données de l'exercice 1 de la séance précédente. Il s'agit de la moyenne générale et des moyennes des trois UE du semestre 3 (2017-2018) de 5 étudiants tirés au hasard.

| Informatique  | Culture scientifique, sociale et humaine | Méthodologie et projets  | Moyenne générale|
|---|---|---|---|
| 10.83  | 12.43  |  14.88 |  12.28 |
|  12.22 | 13.69  | 11.92  |  12.75 |
| 9.84  | 10.88  | 15.38  |  11.36 |
|  12.81 | 12.12  | 15.21  |  13.02 |
| 10.31  |  10.88 | 16.42  |  11.76 |

*On redonne ci-dessous les définitions des matrices dont nous aurons besoin par la suite.*

In [ ]:
M = array([[10.83,12.43 ,14.88 ,12.28],
          [12.22 ,13.69 ,11.92,12.75],
          [9.84 ,10.88 ,15.38 ,11.36],
          [12.81 ,12.12 ,15.21 ,13.02],
          [10.31 ,10.88 ,16.42 ,11.76]])
print(M)
n,p = shape(M)
Z = zeros((n,p))
for i in range(p):
    Z[:,i]=(M[:,i]-mean(M[:,i]))/std(M[:,i])
R=1/n*dot(Z.T,Z)
print(R)

**Question 1.** Calculer les matrices $P$ et $D$ telles que $R=PDP^{-1}$, où

- $R$ est la matrice de corrélation définie précédemment
- $D=diag(\lambda_1,\lambda_2,\lambda_3,\lambda_4)$, avec $\lambda_1\geq\lambda_2\geq\lambda_3\geq\lambda_4$
- $^t\!P=P^{-1}$

In [ ]:
# correction
valtemp, Ptemp =eigh(R)
val = sort(valtemp)[::-1]
indexsort = argsort(valtemp)[::-1]
P=zeros((4,4))
for i in range(4):
    P[:,i]=Ptemp[:,indexsort[i]];
# verification
D=diag(val)
print(R-dot(P,dot(D,inv(P))))
print(D)
print(val)

**Question 2.** Calculer la somme des valeurs propres. Commenter.

In [ ]:
# correction
print(sum(val),trace(D),p)
print("On retrouve bien le nombre de variables (4)")

**Question 3.** Calculer la matrice des composantes principales notée $C$, c'est-à-dire la matrice des individus dans la base des vecteurs propres. Puis représenter les individus dans le plan principal, c'est-à-dire le plan défini par les deux vecteurs propres correspondant aux deux plus grandes valeurs propres.

In [ ]:
# correction
C = dot(Z,P) # composantes principales
scatter(C[:,0], C[:,1]) # individus dans le plan principal
show()

**Question 4.** Calculer la moyenne des composantes principales ainsi que la matrice de covariance de la matrice des composantes principales. Commenter les résultats.

In [ ]:
# correction
m=mean(C,axis=0)
print("la moyenne des composantes principales:",m)
print("la matrice de covariance des composantes principales:")
print(1/n*dot(C.T,C))


# Exercice 3 : programmation d'une ACP 

Le but de cet exercice est de programmer un outil tout prêt pour effectuer une ACP. Pour tester pas à pas les fonctions que nous allons définir, nous allons utiliser les données du fichier `demographie.xls` qui contient des statistiques d'une étude de 1987 concernant 166 pays.
Les statistiques des pays correspondent aux 15 variables (1 qualitative, 14 quantitatives) suivantes :
- COUNTRY : nom du pays
- POP87 : population en 1987
- NAT : taux de natalité
- MORT : taux de mortalité
- ACCR : taux d'accroissement de la population
- POP00 : population prévue pour 2000
- POP20 : population prévue pour 2020
- MORTI : taux de mortalité infantile
- FERTI : taux de fertilité
- AGE15 : proportion de la population  de moins de 15 ans
- AGE65 : proportion de la population  de plus de 65 ans
- ESPER : espérance de vie
- URBA : taux d'urbanisation
- PNB : produit national brut
- CONTI : continent (1=Afrique, 2=Asie, 3=Amérique, 4=Europe, 5=Océanie)

In [ ]:
df =read_excel('demographie.xls')
df.head()

On définit les arrays suivants:

-pays : les noms des pays

-continent : les numéros des continents

-data : les autres variables (qui sont quantitatives et donc adaptées à une ACP), ainsi que leurs labels.


In [ ]:
pays = df['COUNTRY'].values
continent = df['CONTI'].values
data = df[['POP87','NAT','MORT','ACCR','POP00','POP20','MORTI','FERTI','AGE15','AGE65','ESPER','URBA','PNB']].values
labels=['87','NAT','MORT','ACCR','00','20','MORTI','FERTI','-15','+65','ESPER','URBA','PNB']

**Question 1.** Ecrire une fonction `normalise` qui, étant donné un tableau de données, normalise (centre et réduit) chaque colonne du tableau. Testez votre fonction sur la matrice `data`.

In [ ]:
# correction
def normalise(M):
    n,p=M.shape
    Z=zeros((n,p))
    for i in range(p):
        Z[:,i]=(M[:,i]-mean(M[:,i]))/std(M[:,i])# on normalise chaque colonne
    return Z
#print(normalise(data))

**Question 2.** Ecrire une fonction `correlation` qui, étant donné un tableau de données (en ligne les individus, en colonne les variables), renvoie sa matrice de corrélation.  Testez votre fonction sur `data`.

In [ ]:
# correction
def correlation(M):
    Z= normalise(M)
    n=Z.shape[0];
    return 1/n*dot(Z.T,Z)
#print(correlation(data))

**Question 3.** En réutilisant les commandes écrites précédemment, construire une fonction `acp` qui prend en entrée un tableau de données et qui renvoie les valeurs principales ordonnées de manière décroissante, les vecteurs principaux et les composantes principales correspondantes. Testez votre fonction avec `data`.

In [ ]:
# correction
def acp(M):
    n,p=M.shape
    R= correlation(M)
    valtemp, vectemp = eigh(R) # valeurs propres et vecteurs propres de la matrice de corrélation
    val = sort(valtemp)[::-1] # valeurs propres par ordre décroissant
    index = argsort(valtemp)[::-1] # indice du réarrangement des valeurs propres par ordre décroissant
    P=zeros((p,p)) # matrice de changement de base ordonnée en fonction des valeurs propres
    for i in range(p):
        P[:,i]=vectemp[:,index[i]]
    C=dot(normalise(M),P)
    return val, P, C
#val,P, C = acp(data)
#print(val)

**Question 4.** Vérifiez pour la matrice `data` que la matrice de covariance des composantes principales est une matrice diagonale dont les termes diagonaux sont les valeurs principales ordonnées de manière décroissante.

In [ ]:
# correction
n,p = C.shape
print(1/n*dot(C.T,C))

**Question 5.** À partir de `data`, représenter les points-individus dans le plan principal en nommant les axes (`CP 1`, `CP 2`).  

In [ ]:
# correction 
figure(figsize=(12,8)) # taille figure
scatter(C[:,0],C[:,1])
xlabel('CP1')
ylabel('CP2')
show()

**Question 6.** Reprendre le graphique précédent en attribuant une couleur spécifique à l'individu en fonction de la variable `continent`. Commenter.

In [ ]:
# correction 
figure(figsize=(12,8)) # taille figure
for i in range(n):
    if (continent[i]==1):
        scatter(C[i,0],C[i,1], color='red', label='Afrique');
    elif (continent[i]==2):
        scatter(C[i,0],C[i,1], color='C0', label='Asie');
    elif (continent[i]==3):
        scatter(C[i,0],C[i,1], color='green', label='Amerique');
    elif (continent[i]==4):
        scatter(C[i,0],C[i,1], color='yellow', label='Europe');
    else:
        scatter(C[i,0],C[i,1], color='black', label='Océanie');
xlabel('CP1')
ylabel('CP2')
show()

**Question 7.** Pour comparer, représenter les points-individus dans un plan de votre choix (en choisissant deux variables parmi les variables initiales), toujours en attribuant une couleur spécifique à l'individu en fonction de la variable `continent`. Commenter.

In [ ]:
# correction
figure(figsize=(12,8)) # taille figure
ii=3 #je choisis la variable `ACCR` pour l'axe des abscisses
jj=4 #je choisis la variable `00` pour l'axe des ordonnées
for i in range(n):
    if (continent[i]==1):
        scatter(data[i,ii],data[i,jj], color='red', label='Afrique');
    elif (continent[i]==2):
        scatter(data[i,ii],data[i,jj], color='C0', label='Asie');
    elif (continent[i]==3):
        scatter(data[i,ii],data[i,jj], color='green', label='Amerique');
    elif (continent[i]==4):
        scatter(data[i,ii],data[i,jj], color='yellow', label='Europe');
    else:
        scatter(data[i,ii],data[i,jj], color='black', label='Océanie');
xlabel('ACCR')
ylabel('00')
show()